In [44]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import pandas as pd
import os
import glob
import copy
from pathlib import Path

from pprint import pprint
import itertools as it
from collections import Counter
import numpy as np

from aiida import load_profile
from aiida.orm.nodes.data.structure import StructureData

from aiida_quantumespresso.tools.pwinputparser import create_builder_from_file
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.calculations.pwimmigrant import PwimmigrantCalculation
from aiida.engine import submit
from aiida_quantumespresso.calculations.pw import PwCalculation
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida.orm import load_node, load_code

from qe_tools.exceptions import ParsingError

# from own_utils.calc_df import init_project_df

from pymatgen.io.ase import AseAtomsAdaptor

# import pybat
# from pybat import Cathode, LiRichCathode, Dimer, DimerNEBAnalysis

from ase.visualize import view
from ase.io.vasp import read_vasp, write_vasp
from ase.build.tools import sort

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
pd.set_option('display.max_colwidth', None)

In [45]:
load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [46]:
# iurii_olivine_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/LixMnPO4/Li1.00/DFT_plus_U/1_vcrelax/'
# iurii_olivine_file = 'LiMnPO4.vcrelax.1.in'
# iurii_pseudo_dir = '/home/jgeiger/projects/bat_uv_ml/data/olivines_iurii/Pseudopotentials/SSSP'
# iurri_spinel_dir = '/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U/LiMn2O4/1_vcrelax'
# iurii_spinel_file = 'LiMn2O4.vcrelax.1.in'

In [47]:
spinel_dir_iurii = '/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod'
olivine_dir_iurii = '/home/jgeiger/projects/bat_uv_ml/data/olivine_iurii_unmod'
project_dir = '/home/jgeiger/projects/bat_uv_ml'
pseudo_dir = '/usr/share/espresso/pseudo/'

In [48]:
def init_project_df(calc_dir, file_glob):
    # TODO: Extent for multiple file types, also for output files -> can be done by passing a tuple, which endswith accepts.
    # path_list = [os.path.dirname(path) for path in Path(calc_dir).rglob(file_glob)]
    path_list = []
    for root, dirs, files in os.walk(calc_dir):
        for file in files:
            if file.endswith(file_glob):
                path_list.append(os.path.join(root, file))
    # path_list = [_.replace('+', '\+') for _ in path_list]

    full_project_df = pd.DataFrame()
    full_project_df['abs_path'] = [os.path.dirname(_) for _ in path_list]
    full_project_df['calc_in'] = [os.path.basename(_) for _ in path_list]
    # ! Rel path buggy because it ends up being 
    # full_project_df['rel_path'] = full_project_df['abs_path'].str.replace(project_dir, '.')

    return full_project_df

def add_calcdata(df_in):
    calctype_regex = r"\.(pdos|dos|nscf|scf|hp|vcrelax)\."
    df_out = df_in.copy(deep=True)
    df_out['calc_name'] = df_out['calc_in'].apply(lambda x: Path(x).stem)
    df_out['calc_type'] = df_out['calc_in'].str.extract(calctype_regex)
    df_out['calc_out'] = df_out['calc_in'].str.replace('.in', '.out', regex=False)
    return df_out

# ! Also matches paremeters.in file, which results in calctype = nan
spinel_df_iurii = init_project_df(
    calc_dir=spinel_dir_iurii,
    file_glob='.in' # ('.in', '.out')
    )

olivine_df_iurii = init_project_df(
    calc_dir=olivine_dir_iurii,
    file_glob='.in'
    )
# olivine_df_iurii = olivine_df_iurii.dropna()


In [49]:
iurii_full_data_df = pd.concat([spinel_df_iurii, olivine_df_iurii], ignore_index=True)
iurii_full_data_df = add_calcdata(iurii_full_data_df)

iurii_full_data_df.shape
iurii_full_data_df.head()

# pandas write to csv
iurii_full_data_df.to_csv(os.path.join(project_dir, 'data', 'iurii_full_data_df.csv'), index=False)


(1304, 5)

,abs_path,calc_in,calc_name,calc_type,calc_out
0,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.1.in,LiMn1.5Ni0.5O4.scf.1,scf,LiMn1.5Ni0.5O4.scf.1.out
1,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.2.in,LiMn1.5Ni0.5O4.scf.2,scf,LiMn1.5Ni0.5O4.scf.2.out
2,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.dos.in,LiMn1.5Ni0.5O4.dos,dos,LiMn1.5Ni0.5O4.dos.out
3,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.pdos.in,LiMn1.5Ni0.5O4.pdos,pdos,LiMn1.5Ni0.5O4.pdos.out
4,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii_unmod/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax,LiMn1.5Ni0.5O4.vcrelax.in,LiMn1.5Ni0.5O4.vcrelax,vcrelax,LiMn1.5Ni0.5O4.vcrelax.out


# Read in fully lithiated phospho-olivine and spinel structures from pw inputs

In [50]:
print(spinel_df_iurii['rel_path'][0])

./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos


In [65]:

# ! Way too complex with builder, do it via ase reading in QE input
def add_structure_data(df_in):
    df_out = df_in.copy(deep=True)
    # iterate over tuples of dataframe rows
    structure_datas = []
    for index, row in df_out.iterrows():
        # df_out.at[index, 'structureData'] = 
        try:
            created_builder = create_builder_from_file(
                input_folder=row['abs_path'],
                input_file_name=row['filename'],
                code='qe-6.4.1-pw',
                metadata={},
                pseudo_folder_path=pseudo_dir
                )
            structure_data = created_builder['structure']
        except FileNotFoundError:
            # ! Missing PP (for Ni, PBEsol USPP)
            structure_data = None
        except ParsingError:
            # ! No atomic coordinates in input file, e.g. for DOS
            structure_data = None
        except ValueError:
            print(repr(row['abs_path']))
            print(row['filename'])
            structure_data = None
            raise
        
        structure_datas.append(structure_data)
    df_out['structuredata'] = structure_datas 
    return df_out
    
devel_df = spinel_df_iurii.loc[spinel_df_iurii['abs_path'].str.contains('LixMn2O4/DFT\+U/LiMn2O4/1_vcrelax')]
devel_df = spinel_df_iurii.copy(deep=True)
devel_df.shape
print(devel_df['abs_path'])

devel_df = add_structure_data(devel_df)

devel_df.head()

# devel_df['structureData'] = devel_df.apply(
#     lambda df: create_builder_from_file(
#         input_folder=df['abs_path'],
#         input_file_name=df['filename'],
#         code='qe-6.4.1-pw',
#         metadata={},
#         pseudo_folder_path=pseudo_dir
#         ),
#     axis=1
# )
# devel_df.head()

(235, 4)

0         /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos
1         /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos
2         /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos
3         /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos
4      /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax
                                                     ...                                                
230                   /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/2_calcUV
231                   /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/2_calcUV
232                   /home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/2_calcUV
233                  /home/jgeiger/projects/bat_uv_ml/d

,abs_path,filename,rel_path,calctype,structuredata
0,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.1.in,./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,scf,None
1,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.scf.2.in,./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,scf,None
2,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.dos.in,./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,dos,None
3,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,LiMn1.5Ni0.5O4.pdos.in,./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/5_pdos,pdos,None
4,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax,LiMn1.5Ni0.5O4.vcrelax.in,./data/spinel_iurii/LixMn1.5Ni0.5O4/DFT+U/LiMn1.5Ni0.5O4/3_vcrelax,vcrelax,None


In [66]:

devel_df.tail(3)

,abs_path,filename,rel_path,calctype,structuredata
232,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/2_calcUV,Mn2O4.scf.1.in,./data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/2_calcUV,scf,uuid: 854861a1-d1ce-41da-a711-a259c9e7514f (unstored)
233,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/1_vcrelax,Mn2O4.vcrelax.2.in,./data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/1_vcrelax,vcrelax,uuid: 1417b7b8-e442-4ba0-a758-ec9188f40284 (unstored)
234,/home/jgeiger/projects/bat_uv_ml/data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/1_vcrelax,Mn2O4.vcrelax.1.in,./data/spinel_iurii/LixMn2O4/DFT+U+V/Mn2O4/1_vcrelax,vcrelax,uuid: 80d19952-6ed7-4df9-aea4-3fa2677f29a9 (unstored)


KeyError: 'Structuredata'

***
### Bash cell for verdi stuff

In [78]:
%%bash
# verdi process show 2171
# verdi process report 2171
verdi process list -a -p 1
# aiida load node
verdi computer list

PK    Created    Process label    Process State    Process status
----  ---------  ---------------  ---------------  ----------------

Total results: 0

Report: last time an entry changed state: 2D ago (at 15:11:19 on 2023-02-13)
Report: Using 0% of the available daemon worker slots.
Report: List of configured computers
Report: Use 'verdi computer show COMPUTERLABEL' to display more detailed information
* lumi-small
* theospc24


In [79]:
%%bash
# verdi calcjob inputls 2176    # shows the list of input files
# verdi calcjob inputcat 2176   # shows the input file of the calculation
# verdi calcjob outputls 2176   # shows the list of output files
# verdi calcjob outputcat 2176  # shows the output file of the calculation
# verdi calcjob res 2176        # shows the parser results of the calculation